In [11]:
import os
import json
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spint
import scipy.interpolate as spinter
import matplotlib
import matplotlib.animation as animation
from matplotlib.widgets import Slider

$$
\frac{d \vec{v}}{dt} = \frac{1}{r^3} \left[-M \vec{r} + \vec{v}\times\vec{S} - \frac{3 (\vec{S}\cdot\vec{r})}{r^2} \vec{v}\times\vec{r}\right]
$$

In [28]:
pnts = np.linspace(-6,6,100)
def contract_around_middle(pnts,pow):
    rect = [np.min(pnts),np.max(pnts)-np.min(pnts)]
    nps = 2*(pnts-rect[0])/rect[1] - 1
    nps = np.sign(nps)*np.abs(nps)**pow
    return (nps+1)/2*rect[1]+rect[0]

pnts = contract_around_middle(pnts,3)
plt.scatter(pnts,np.zeros_like(pnts),marker=".")
plt.show()

In [29]:
coord_names = [r'$t$',r'$x$',r'$y$',r'$z$',r'$\frac{d t}{d \tau}$',r'$\frac{d x}{d \tau}$',r'$\frac{d y}{d \tau}$',r'$\frac{d z}{d \tau}$']
def acc_lense_thirring(tau, zs):
    global S, M, R
    a = np.zeros(8)
    rs = zs[1:4]
    vs = zs[5:]
    r = np.sqrt(np.sum(rs*rs))
    if r < R:
        a[0] = 1
        return a
    a[:4] = zs[4:]
    a[5:] -= M*rs
    a[5:] += np.cross(vs,[0,0,S])
    a[5:] -= 3*S*rs[2]/(r*r) * np.cross(vs,rs)
    a[5:] /= r*r*r
    return a

In [30]:
def get_points_on_gridlines(fR, tR, N, count):
    pnts = []
    dr = (np.array(tR)-np.array(fR))/np.array(N)
    ys = np.linspace(fR[1],tR[1],np.max(N)*count)
    for xi in range(N[0]+1):
        xs = np.ones(np.max(N)*count)*(fR[0]+xi*dr[0])
        #pnts = np.append(pnts, [xs, ys], axis=1)
        pnts.append(np.transpose([xs,ys]))
    xs = np.linspace(fR[0],tR[0],np.max(N)*count)
    for yi in range(N[1]+1):
        ys = np.ones(np.max(N)*count)*(fR[1]+yi*dr[1])
        #pnts = np.append(pnts, [xs, ys], axis=1)
        pnts.append(np.transpose([xs,ys]))
    return np.array(pnts)
def get_points_on_cube_grid(fR,tR,N,count):
    pnts = []
    

In [31]:
def generate_3d_grid_lines(start_point, end_point, num_lines, subdivisions):
    """
    Generate lines on a 3D grid.

    Parameters:
    - start_point: A tuple (x0, y0, z0) representing the starting point of the grid.
    - end_point: A tuple (x1, y1, z1) representing the ending point of the grid.
    - num_lines: A tuple (nx, ny, nz) representing the number of lines in each direction.
    - subdivisions: An integer representing the number of subdivisions per line.

    Returns:
    - lines: A list of lists, where each inner list contains the points of a specific line.
    """
    # Unpack the start and end points
    x0, y0, z0 = start_point
    x1, y1, z1 = end_point
    
    # Unpack the number of lines in each direction
    nx, ny, nz = num_lines
    
    # Generate the grid lines in each direction
    x_lines = np.linspace(x0, x1, nx)
    y_lines = np.linspace(y0, y1, ny)
    z_lines = np.linspace(z0, z1, nz)
    
    # Generate the points on each line with the specified number of subdivisions
    lines = []
    
    # Lines parallel to the yz-plane
    for x in x_lines:
        for y in y_lines:
            z_points = contract_around_middle(np.linspace(z0, z1, subdivisions),3)
            line = np.column_stack((np.full(subdivisions, x), np.full(subdivisions, y), z_points))
            lines.append(line.tolist())
    
    # Lines parallel to the xz-plane
    for x in x_lines:
        for z in z_lines:
            y_points = contract_around_middle(np.linspace(y0, y1, subdivisions),3)
            line = np.column_stack((np.full(subdivisions, x), y_points, np.full(subdivisions, z)))
            lines.append(line.tolist())
    
    # Lines parallel to the xy-plane
    for y in y_lines:
        for z in z_lines:
            x_points = contract_around_middle(np.linspace(x0, x1, subdivisions),3)
            line = np.column_stack((x_points, np.full(subdivisions, y), np.full(subdivisions, z)))
            lines.append(line.tolist())
    
    return np.array(lines)


In [32]:
def get_geodesic(tau_max, z0, t_evals, tol=1e-6):
    rkdp = spint.RK45(acc_lense_thirring, 0.0, z0,tau_max,first_step=1e-8,rtol=tol,atol=tol*1e-2)
    taus = [0.0]
    zs = [rkdp.y]
    while rkdp.y[0] < tau_max and rkdp.t < tau_max:
        rkdp.step()
        taus.append(rkdp.t)
        zs.append(rkdp.y)
        if rkdp.status != "running":
            break
    zs = np.array(zs).T
    intf = spinter.interp1d(taus, zs, fill_value=(zs[:,0],zs[:,-1]), bounds_error=False)
    return intf(t_evals)

In [37]:
all_setups = {}
if os.path.isfile('all_setups.json'):
    try:
        with open('all_setups.json', 'r') as fp:
            all_setups = json.load(fp)
    except:
        print("could not open all_setups.json")

setup = {
    "spawn_pos" : [0,0,0],
    "grid_size" : [12],
    "spawn_vel" : [0,0,0],
    "num_lines" : [6,6,6],
    "subdivisions": 200,
    "tau_max" : 10.0,
    "M" : 1.0,
    "R" : 1.0,
    "omega" : 1e1*np.pi,
}
setupnum = 0.5
save_setup = True
#setup = all_setups[f"setup {setupnum}"]

if save_setup:
    all_setups[f"setup {setupnum}"] = setup
    with open('all_setups.json', 'w') as fp:
        json.dump(all_setups, fp)
else:
    setup = all_setups[f"setup {setupnum}"]
print(setup)

{'spawn_pos': [0, 0, 0], 'grid_size': [12], 'spawn_vel': [0, 0, 0], 'num_lines': [6, 6, 6], 'subdivisions': 200, 'tau_max': 10.0, 'M': 1.0, 'R': 1.0, 'omega': 31.41592653589793}


In [ ]:
M = setup["M"]
R = setup["R"]
S = 2.0/5.0 * M * R**2 * setup["omega"]
lines = generate_3d_grid_lines(np.array(setup["spawn_pos"])-np.array(setup["grid_size"])/2, np.array(setup["spawn_pos"])+np.array(setup["grid_size"])/2, setup["num_lines"], setup["subdivisions"])
print(lines.shape)

taus = np.linspace(0,setup["tau_max"],100)
line_ev = np.empty((len(lines), len(taus), len(lines[0]), 3))
#for li,line in enumerate(lines):
for li in tqdm(range(len(lines))):
    line = lines[li]
    for pi,pnt in enumerate(line):
        z0 = np.array([0,pnt[0],pnt[1],pnt[2],1,setup["spawn_vel"][0],setup["spawn_vel"][1],setup["spawn_vel"][2]])
        #print(f"calculating trajectory for point {pi} on line {li} with z0={np.array2string(z0,precision=3)}.")
        zs = get_geodesic(setup["tau_max"], z0,taus,1e-6)
        line_ev[li,:,pi,:] = zs[1:4].T

print(line_ev.shape)
np.save(f'./assets/spacetime_sims/spacetime_lines{setupnum}.npy', line_ev)
np.save(f'./assets/spacetime_sims/lines{setupnum}_timestamps.npy', taus)

(108, 200, 3)


100%|██████████| 108/108 [05:56<00:00,  3.30s/it]

(108, 100, 200, 3)


In [ ]:
line_ev = np.load(f'./assets/spacetime_sims/spacetime_lines{setupnum}.npy')
taus = np.load(f'./assets/spacetime_sims/lines{setupnum}_timestamps.npy')
print(line_ev.shape)
print(taus.shape)

(108, 100, 200, 3)
(100,)


In [43]:
%matplotlib qt

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

fig.subplots_adjust(bottom=0.25)

ax.set_aspect('equal')
ax.set(xlim=[-8,8], ylim=[-8,8], zlim3d=[-8,8], xlabel=r'x', ylabel=r'y')
ax.view_init(elev=10., azim=(-100))

def draw_sphere(phase=0.0, radius=1.0):
    u, v = np.mgrid[0:2*np.pi:14j, 0:np.pi:7j]
    x = radius*np.cos(u+phase)*np.sin(v)
    y = radius*np.sin(u+phase)*np.sin(v)
    z = radius*np.cos(v)
    return ax.plot_surface(x, y, z, color="red", zorder=-10)

sphere = draw_sphere(0,1)

line_objs = []
for line in line_ev[:,0,:,:]:
    pnts = line.T
    [line_obj] = ax.plot(pnts[0],pnts[1],pnts[2],color="k",zorder=10)
    line_objs.append(line_obj)

tau_slider_ax  = fig.add_axes([0.25, 0.15, 0.65, 0.03])
tau_slider = Slider(tau_slider_ax, 'tau idx', 0, len(taus)-1, valstep=1, valinit=0)

angle_slider_ax = fig.add_axes([0.25, 0.1, 0.65, 0.03])
angle_slider = Slider(angle_slider_ax, 'angle', 0, 360, valinit=0)

def tau_on_changed(val):
    tau_idx = tau_slider.val
    tau = taus[tau_idx]
    fig.suptitle(rf'$\tau={tau:0.2f}$')
    for li,line in enumerate(line_ev[:,tau_idx,:,:]):
        line_objs[li].set_data_3d(line_ev[li,tau_idx,:,:].T)
    fig.canvas.draw_idle()
def angle_on_changed(val):
    angle = angle_slider.val
    ax.view_init(elev=10., azim=(-angle-100))
    fig.canvas.draw_idle()
tau_slider.on_changed(tau_on_changed)
angle_slider.on_changed(angle_on_changed)

plt.show()

In [34]:
%matplotlib qt


fig = plt.figure()
ax = fig.add_subplot(projection='3d')

fig.subplots_adjust(bottom=0.25)

ax.set_aspect('equal')
ax.set(xlim=[-8,8], ylim=[-8,8], zlim3d=[-8,8], xlabel=r'x', ylabel=r'y')
ax.view_init(elev=10., azim=(-100))

def draw_sphere(phase=0.0, radius=1.0):
    u, v = np.mgrid[0:2*np.pi:14j, 0:np.pi:7j]
    x = radius*np.cos(u+phase)*np.sin(v)
    y = radius*np.sin(u+phase)*np.sin(v)
    z = radius*np.cos(v)
    return ax.plot_surface(x, y, z, color="red", zorder=-10)

sphere = draw_sphere(0,1)

pnt_objs = []
for line in line_ev[:,90,:,:]:
    pnts = line.T
    pnt_obj = ax.scatter(pnts[0],pnts[1],pnts[2], marker=".",color="k",zorder=10)
    pnt_objs.append(pnt_obj)

tau_slider_ax  = fig.add_axes([0.25, 0.15, 0.65, 0.03])
tau_slider = Slider(tau_slider_ax, 'tau idx', 0, len(taus)-1, valstep=1, valinit=0)

angle_slider_ax = fig.add_axes([0.25, 0.1, 0.65, 0.03])
angle_slider = Slider(angle_slider_ax, 'angle', 0, 360, valinit=0)

def tau_on_changed(val):
    tau_idx = tau_slider.val
    tau = taus[tau_idx]
    fig.suptitle(rf'$\tau={tau:0.2f}$')
    for li,line in enumerate(line_ev[:,tau_idx,:,:]):
        pnt_objs[li].set_offsets(line_ev[li,tau_idx,:,:].T)
    fig.canvas.draw_idle()
def angle_on_changed(val):
    angle = angle_slider.val
    ax.view_init(elev=10., azim=(-angle-100))
    fig.canvas.draw_idle()
tau_slider.on_changed(tau_on_changed)
angle_slider.on_changed(angle_on_changed)

plt.show()

In [23]:
# %matplotlib inline
# plt.rcParams["animation.html"] = "jshtml"
# #plt.rcParams['figure.dpi'] = 100
# matplotlib.rcParams['animation.embed_limit'] = 2**32
# plt.ioff()

# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')

# def update(frame):
#     ax.cla()
#     ax.set_aspect('equal')
#     ax.set_xlim(0,10)
#     ax.set_ylim(-5,5)
#     ax.set(zlim3d=[-5,5])
#     ax.set_xlabel(r'x')
#     ax.set_ylabel(r'y')
#     for line in line_ev[:,frame,:,:]:
#         pnts = line.T
#         ax.plot(pnts[0],pnts[1],pnts[2],color="k")
#     return

# def set_angle(a):
#     ax.view_init(elev=10., azim=(a-100))

# ani = animation.FuncAnimation(fig=fig, func=update, frames=range(0,100,2), interval=30)

In [24]:
lines = generate_3d_grid_lines((0, 0, 0), (1, 1, 1), (4, 4, 4), 7)
print(lines.shape)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
for line in lines:
    line = line.T
    ax.plot(line[0],line[1],line[2],marker=".",color="k")
plt.show()

(48, 7, 3)
